In [ ]:
!pip install -q -U google-generativeai

In [49]:
import pathlib
import textwrap
import os
import google.generativeai as genai

# store the api keys in user_data.py of directory
from user_data import google_api_key

# Used to securely store your API key
#from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [50]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
#GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
#print(os.environ['GOOGLE_API_KEY'])
genai.configure(api_key=google_api_key)

In [37]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


In [64]:
model = genai.GenerativeModel('gemini-pro')

In [65]:
ingredients = "['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 Tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits']"

In [66]:
# Prompt
prompt_task = """
Using the given ingredients of a recipe, You need to just return MAXIMUM ONE WORD and nothing else whether it is 'NonVeg' or 'Veg' or 'Vegan'. Ingredients: 
"""
#prompt_text = prompt_task + ingredients

In [33]:
#%%time
#response = model.generate_content(prompt_text)

CPU times: total: 15.6 ms
Wall time: 3.31 s


In [34]:
#to_markdown(response.text)

> {
>  "category": "Vegan",
>  "reasoning": "This recipe does not contain any animal-derived ingredients, such as meat, fish, eggs, or dairy products. It is made with plant-based ingredients like brown sugar, evaporated milk (which can be made from soy or other plant-based milks), vanilla, nuts, and rice biscuits.",
>  "nutritional_information": {
>   "calories": 350,
>   "total_fat": 15 grams,
>   "saturated_fat": 5 grams,
>   "cholesterol": 0 milligrams,
>   "sodium": 100 milligrams,
>   "carbohydrates": 50 grams,
>   "dietary_fiber": 5 grams,
>   "sugar": 25 grams,
>   "protein": 5 grams
>  }
> }

Classification Testing on Data

In [67]:
import pandas as pd

In [68]:
df = pd.read_csv("recipe_top_1000_rows.csv")

In [69]:
df2 = df.head(500)

In [70]:
df2['Gemini_OP'] = df2.ingredients.apply(lambda x: model.generate_content(prompt_task+x).text)

C:\Users\mm0450.MAVEN\AppData\Local\Temp\ipykernel_14836\1160561182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Gemini_OP'] = df2.ingredients.apply(lambda x: model.generate_content(prompt_task+x).text)


In [71]:
df2.to_csv("Gemini_raw_output_v3.csv")

In [78]:
# Nutrition Task
prompt_nutrition = "Using the given ingredients of a recipe, You need to just return nutritional information enclosed in double quotations containing comma separated values. Ingredients: "

In [79]:
df2['nutritional_data'] = df2.ingredients.apply(lambda x: model.generate_content(prompt_nutrition+x).text)

C:\Users\mm0450.MAVEN\AppData\Local\Temp\ipykernel_14836\1704469627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['nutritional_data'] = df2.ingredients.apply(lambda x: model.generate_content(prompt_nutrition+x).text)


In [80]:
df2.to_csv("Gemini_raw_output_v6.csv")